In [19]:
import xml.etree.ElementTree as ET
import pandas as pd
import os
from BeautifulSoup import BeautifulSoup
from pymongo import MongoClient
from lxml import etree

In [22]:
#Path of data files
path = r'C:\Anaconda\Galvanize\Minimester2-Project\cars\data\2007'
#Other initializations
client = MongoClient()
db = client.reviews
collection = db.cars

#loop through files
for i,name in enumerate(os.listdir(path)):
    #context manager
    if name in ['Bad']:
        break
    with open(path+'\\'+name, 'r') as fo:
        #read in the data
        data = fo.read().splitlines(True)
        #construct etree XML friendly strings
        newdata = '<DATA>\n'+''.join(data[1:]).replace('&','and')+'</DATA>'
#         print ''.join(data[1:]).replace('&','and')
        soup = BeautifulSoup(newdata)
#         print str(soup)
        root = etree.fromstring(str(soup), 
                        parser=etree.XMLParser(encoding='cp1252'))

        #build lists of data
        date, author, text, favorite, = [],[],[],[]
        #loop through the tree to capture all data
        for j, doc in enumerate(root):
            #conditionals to build lists of data
#             print j
            for child in doc:
#                 print child.tag, child.text
                if child.tag.lower() == 'date':
                    date.append(child.text)
                elif child.tag.lower() == 'author':
                    author.append(child.text)
                elif child.tag.lower() == 'text':
                    text.append(child.text)
                elif child.tag.lower() == 'favorite':
                    favorite.append(child.text)
#         print len(date),len(author),len(text),len(favorite)
        df = pd.DataFrame(zip(date,author,text,favorite))
#         print df.head()
        df.columns = ['Date','Author','Text','Favorite']
        df['Vehicle'] = name
        #Insert df into MongoDB
        result = collection.insert(df.T.to_dict().values())
        #Checks that the data load is proceeding
        print name
        print collection.count()

2007_acura_mdx
170
2007_acura_rdx
335
2007_acura_rl
357
2007_acura_tl
508
2007_acura_tsx
564
2007_audi_a3
604
2007_audi_a4
698
2007_audi_a6
718
2007_audi_a8
736
2007_audi_q7
808
2007_audi_rs4
834
2007_audi_s8
848
2007_bmw_3_series
1113
2007_bmw_5_series
1204
2007_bmw_6_series
1221
2007_bmw_x3
1297
2007_bmw_x5
1341
2007_bmw_z4
1363
2007_buick_lacrosse
1394
2007_buick_lucerne
1481
2007_buick_rainier
1495
2007_buick_rendezvous
1534
2007_cadillac_cts
1572
2007_cadillac_dts
1592
2007_cadillac_escalade
1667
2007_cadillac_escalade_esv
1689
2007_cadillac_escalade_ext
1706
2007_cadillac_srx
1740
2007_cadillac_sts
1762
2007_chevrolet_avalanche
1857
2007_chevrolet_aveo
1934
2007_chevrolet_cobalt
2066
2007_chevrolet_colorado
2111
2007_chevrolet_corvette
2196
2007_chevrolet_equinox
2273
2007_chevrolet_hhr
2393
2007_chevrolet_impala
2547
2007_chevrolet_malibu
2585
2007_chevrolet_malibu_maxx
2599
2007_chevrolet_monte_carlo
2623
2007_chevrolet_silverado_1500
2764
2007_chevrolet_silverado_1500_classic


In [21]:
collection.drop()